# Проект по SQL

# Описание

- Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
- В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Подключение к бд

In [1]:
import pandas as pd 
from sqlalchemy import text, create_engine 

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя 
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas 
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


# Исследуем таблицы

## books

In [5]:
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## autors

In [6]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## publishers

In [7]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## ratings

In [8]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


## reviews

In [9]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Посчитаем, сколько книг вышло после 1 января 2000 года;

In [10]:
query = '''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


- после 1 января 2000 года вышло 819 книг

# Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS cnt_reviews, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id

LEFT JOIN  ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY cnt_reviews DESC

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


В базе имеется 100 книг и как видим на некоторые книги нет обзоров, хотя оценка высока

# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [12]:
query = '''

WITH 
tab AS (
SELECT publisher_id,
       book_id
FROM books
WHERE num_pages > 50
)

SELECT tab.publisher_id,
       p.publisher
FROM tab 

INNER JOIN publishers AS p
ON tab.publisher_id = p.publisher_id

GROUP BY tab.publisher_id, p.publisher
ORDER BY COUNT(tab.book_id) DESC

LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,212,Penguin Books


## Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query = '''

WITH 

tab AS (
SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) > 50)


SELECT a.author_id, 
       a.author, 
       AVG(r.rating) AS avg_rating
FROM books AS b

INNER JOIN tab
ON b.book_id = tab.book_id

INNER JOIN authors AS a
ON b.author_id = a.author_id

INNER JOIN ratings AS r
ON tab.book_id = r.book_id

GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC

LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Автры с самой высокой средней оценкой книг это J.K. Rowling и Mary GrandPré

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = '''

SELECT COUNT(DISTINCT rv.text)/ COUNT(DISTINCT rv.username) AS avg_rv_count
FROM ratings AS r

INNER JOIN reviews AS rv
ON r.username = rv.username

WHERE r.username in ( 
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50
                    )

'''

pd.io.sql.read_sql(query, con = engine)

,avg_rv_count
0,24


Среднее количестиво обзоров от пользоватеелй поставивших более 50 оценок равно 24

In [15]:
query = '''

SELECT EXTRACT(YEAR FROM publication_date) as god,
       COUNT(DISTINCT publisher_id) as cnt_publisher,
       COUNT(book_id) as cnt_books,
       SUM(num_pages) sum_pages
       
FROM books

GROUP BY god
HAVING COUNT(book_id) > 30
ORDER BY sum_pages DESC

'''

pd.io.sql.read_sql(query, con = engine)

,god,cnt_publisher,cnt_books,sum_pages
0,2006.0,109,184,68302
1,2005.0,89,139,55967
2,2004.0,88,124,46779
3,2003.0,65,105,41423
4,2002.0,62,94,38597
5,2001.0,41,60,21758
6,2007.0,38,50,18258
7,1999.0,26,41,15763
8,2000.0,35,38,13328


- как мы видим до 99 года выходило меньше 30 книг

# Вывод:
 - после 1 января 2000 года вышло 819 книг
 - Среднее количестиво обзоров от пользоватеелй поставивших более 50 оценок равно 24
 - Автры с самой высокой средней оценкой книг это J.K. Rowling и Mary GrandPré
 - издательство, которое выпустило наибольшее число книг толще 50 страниц Penguin Books